In [1]:
!pip install transformers accelerate trl bitsandbytes peft
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 17.1 MB/s eta 0:00:00


In [2]:
import torch
import os
from google.colab import userdata
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments,GemmaTokenizer
from peft import LoraConfig
from trl import SFTTrainer

import os
os.environ["HF_TOKEN"]=userdata.get("HF_TOKEN")

In [3]:
model_id="google/gemma-3-270m"
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [6]:
tokenizer=AutoTokenizer.from_pretrained(model_id,token=os.environ["HF_TOKEN"])
model1=AutoModelForCausalLM.from_pretrained(model_id,token=os.environ["HF_TOKEN"])

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/536M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

In [7]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model1 = model1.to(device)


In [8]:
if torch.cuda.is_available():
  print("available")

available


In [9]:
text="Hi who is manu?"
device="cuda:0"
input=tokenizer(text,return_tensors="pt").to(device)
output=model1.generate(**input,max_new_tokens=50)

In [10]:
answer=tokenizer.decode(output[0],skip_special_tokens=True)
print(answer)

Hi who is manu?

1) 27.2mm

2) 451.60mm

3) 807.95mm

4) 977.92mm

5) 844.


In [11]:
os.environ["WANDB_DISABLED"]="false"
lora_config=LoraConfig(
    r=8,
    target_modules=["q_proj","o_proj","k_proj","v_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM"
)

In [66]:
from datasets import Dataset, DatasetDict

# Create 10 example records
data = {
    "description": [
        "He is a Gen AI developer working on NLP projects",
        "He is a machine learning engineer building recommendation systems",
        "He is a backend developer integrating AI APIs for clients",
        "He is a deep learning engineer working on computer vision",
        "He is a software engineer creating AI-powered web applications",
        "He is an AI researcher experimenting with generative models",
        "He is a data engineer designing pipelines for ML workflows",
        "He is a prompt engineer optimizing LLM prompts for efficiency",
        "He is a robotics engineer integrating AI into autonomous systems",
        "He is a cloud engineer managing AI infrastructure and deployments",
        "He is an NLP engineer fine-tuning language models for chatbots",
        "He is a software architect designing scalable AI platforms",
        "He is a DevOps engineer automating AI model deployments",
        "He is a computational linguist analyzing language data with AI",
        "He is a researcher teaching models on specialized AI datasets"
    ],
    "person": [
        "Manu", "Manu", "Manu", "Manu", "Manu",
        "Manu", "Manu", "Manu", "Manu", "Manu",
        "Manu", "Manu", "Manu", "Manu", "Manu"
    ]
}

train_dataset = Dataset.from_dict(data)
dataset = DatasetDict({
    "train": train_dataset
})

In [67]:
dataset['train']

Dataset({
    features: ['description', 'person'],
    num_rows: 15
})

In [68]:
def format_func(example):
    return (
        f"Given the description below, name the person.\n"
        f"Description: {example['description']}\n"
        f"Person: {example['person']}"
    )

In [69]:
import transformers

In [70]:
trainer = SFTTrainer(
    model=model1,
    train_dataset=dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=2,
        num_train_epochs=20,
        warmup_steps=2,
        max_steps=-1,
        learning_rate=5e-5,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        logging_dir="./logs",
        report_to=[],
    ),
    peft_config=lora_config,
    formatting_func=format_func,
)


/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Applying formatting function to train dataset:   0%|          | 0/15 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/15 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/15 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/15 [00:00<?, ? examples/s]

In [71]:
trainer.train()

Step,Training Loss
1,4.617700
2,4.444500
3,4.463200
4,4.305200
5,5.070700
6,4.196400
7,4.883100
8,4.327200
9,4.290200
10,3.986100


TrainOutput(global_step=160, training_loss=1.7378905989229678, metrics={'train_runtime': 86.9914, 'train_samples_per_second': 3.449, 'train_steps_per_second': 1.839, 'total_flos': 5164382346240.0, 'train_loss': 1.7378905989229678, 'epoch': 20.0})

In [105]:
text="Given the description below, name the person.\n Description: He is a machine learning engineer building recommendation systems"
device="cuda:0"
input=tokenizer(text,return_tensors="pt").to(device)
output = model1.generate(
    **input,
    max_new_tokens=100,
    do_sample=False,
    temperature=0.7,
    top_p=0.9
)


In [106]:
answer=tokenizer.decode(output[0],skip_special_tokens=True)
print(answer)

Given the description below, name the person.
 Description: He is a machine learning engineer building recommendation systems
 Person: Manu


In [ ]:
    "description": [
        "He is a Gen AI developer working on NLP projects",
        "He is a machine learning engineer building recommendation systems",
        "He is a backend developer integrating AI APIs for clients",
        "He is a deep learning engineer working on computer vision",
        "He is a software engineer creating AI-powered web applications",
        "He is an AI researcher experimenting with generative models",
        "He is a data engineer designing pipelines for ML workflows",
        "He is a prompt engineer optimizing LLM prompts for efficiency",
        "He is a robotics engineer integrating AI into autonomous systems",
        "He is a cloud engineer managing AI infrastructure and deployments",
        "He is an NLP engineer fine-tuning language models for chatbots",
        "He is a software architect designing scalable AI platforms",
        "He is a DevOps engineer automating AI model deployments",
        "He is a computational linguist analyzing language data with AI",
        "He is a researcher teaching models on specialized AI datasets"
    ],